# Práctica 2: Aprendizaje y selección de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

* Mª Mercedes Guijarro


En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

# 1. Preliminares

En primer lugar, importamos todas las librerías necesarias:

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Local application
import miner_a_de_datos_aprendizaje_modelos_utilidad as utils

import pandas as pd

Además, fijamos una semilla para que los experimentos sean reproducibles:

In [ ]:
random_state = 27912

# 2. Pima Indians Diabetes

### 2.1. Carga de datos

Como siempre, en primer lugar, cargamos el conjunto de datos y comprobamos que se haya realizado correctamente:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"


data = pd.read_csv(filepath, dtype={"Outcome": 'category'})

Comprobando que se ha cargado correctamente:

In [ ]:
data.sample(5, random_state=random_state)

Dividimos el conjunto en variables predictoras y la variable clase:

In [ ]:
target = "Outcome"

(X, y) = utils.divide_dataset(data, target)

Vamos a comprobar que se ha separado correctamente. Comenzamos con las variables predictoras:

In [ ]:
X.sample(5, random_state=random_state)

Y continuamos con la variable clase:

In [ ]:
y.sample(5, random_state=random_state)

A continuación, dividimos el conjunto de datos en entrenamiento y test, para de esta forma trabajar sobre el conjunto de entrenamiento y la validación sobre el test. Realizamos la división mediante un holdot estratificado.

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      train_size=train_size,
                                                      random_state=random_state)

Comprobamos que se ha realizado correctamente, tanto para el conjunto de entrenamiento como para el test:

In [ ]:
X_train.sample(5, random_state=random_state)

In [ ]:
y_train.sample(5, random_state=random_state)

In [ ]:
X_test.sample(5, random_state=random_state)

In [ ]:
y_test.sample(5, random_state=random_state)

Y una vez realizada la división pasamos a la creación de modelos para nuestro conjunto de datos.

### 2.2. Obtención de modelos

Partiendo del preprocesamiento que realizamos en la práctica anterior, realizaremos la obtención y selección de modelos mediante un proceso de GridSearch, de esta forma obtendremos la mejor configuración de hiperparámetros para cada uno de ellos y así selección el clasificador que mejor se adapte a nuestro problema.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Value Imputer
imp = SimpleImputer(missing_values=0, strategy='mean')

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('missing', imp ,[False, True, True, True, True, True, True, True]) 
    ], remainder="passthrough")

# Fitting the column transformer
preprocessor = preprocessor.fit(X_train)

#Las columnas a las que no se le aplica la transformación se colocan al final del DataFrame
X_train = pd.DataFrame(preprocessor.transform(X_train), 
                                columns=X_train.columns[1:].append(X_train.columns[:1]))


X_train.describe()

#### 2.2.1. K-Nearest Neighbours (Vecinos más cercanos)

Como hemos visto, la configuración de los hiperparámetros de este algoritmo son principalmente dos: `n_neighbours` y `weights`.

Inicializamos el clasificador:

In [ ]:
n_neighbors = 5

k_neighbors_model = KNeighborsClassifier(n_neighbors)

En el caso del parametro `weights` sólo acepta dos opciones `uniform` o `distance`.

In [ ]:
weights = ['uniform', 'distance']

Por otro lado, la elección del parámetro `n_neighbours`  depende mucho de la distribución de los datos. Una buena manera de seleccionar los valores podría ser fijarlo a $ \sqrt{N} $ o probar los valores comprendidos entre 1 y $ \sqrt{N} $ , pero al probar todas esas configuraciones corremos el riesgo de obtener un modelo sobreajustado, por lo que vamos a probar con `[1, 3, 5, 7, 10]`

In [ ]:
n_neighbors = [1, 3, 5, 7, 10]

Una vez establecidos los valores de los parámetros, utilizamos el algoritmo *GridSearch* para realizar una búsqueda, evaluando mediante validación cruzada todas las posibles combinaciones existentes y seleccionar la mejor.

In [ ]:
estimator = k_neighbors_model

k_neighbors_clf = utils.optimize_params(estimator,
                                        X_train, y_train, cv=5,
                                        weights=weights,
                                        n_neighbors=n_neighbors)

La mejor configuración de hiperparámtros que hemos obtenido es para `n_neighbours` 10 y `weights`  = uniform, es decir que todos los vecinos más cercanos tienen la misma importancia o el mismo peso.
La tasa de acierto obtenida es de $ 0.741 \pm 0.031 $.

#### 2.2.2. Árboles de decisión

Continuando con los árboles de decisión,si analizamos un poc sus hiperparámetros podemos ver que hay una gran parte de ellos que nos sirven para controlar el crecimiento del árbol de decisión.
Ahora, `scikit-learn` incluye un nuevo hiperparámetro que nos permite establecer un umbral a partir del cual se puede realizar una poda (`ccp_alpha`). Podemos obtener valores para este hiperparámetro utilizando el método `cost_complexity_pruning_path`, que nos devuelve todos los valores efectivos obtenidos durate el proceso de poda.


Establecemos por defecto un árbol totalmente profundo:

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

Para los árboles de decisión vamos a tomar los siguientes parámetros:

* Criterion: para medir la calidad de una partición, puede tomar los valores gini o entropy.
* max_depth: altura máxima del árbol. Para este caso vamos a establecer los valores entre el 1 y el 5.

In [ ]:
estimator = clone(decision_tree_model)

criterion = ["gini", "entropy"]
max_depth = [1, 2, 3, 4, 5]
ccp_alpha = [0.0, 0.01, 0.02, 0.03, 0.04]

decision_tree_clf = utils.optimize_params(estimator,
                                          X_train, y_train, cv=5,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha)

Como podemos observar los mejores hiperparámetros obtenidos son para ccp_alpha = 0.01, el criterio gini y para max_depth 4.
La tasa de acierto obtenida es de $ 0.755 \pm 0.0485 $, no es mucho mejor que la obtenida con knn.

#### 2.2.2. AdaBoost (Adaptative Boosting)

Para AdaBoost, implementado como `AdaBoostClassifier` en `scikit-learn`, podemos configurar los siguientes hiperparámetros:
* `base_estimator`: es el estimador base que utilizaremos para la construcción del ensemble.
* `n_estimators`: número de estimadores del ensemble.
* `learning_rate`: coeficiente que se aplica a la *importancia* de los clasificadores a la hora de hacer la predicción.
* `random_state`: semila para la reproducibilidad de los experimentos.

Primero configuramos un modelo AdaBoost sencillo que utilice los hiperparámetros por defecto:

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

Y ahora obtenemos la mejor configuración:

In [ ]:
estimator = adaboost_model

# Should not modify the base original model
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2]
ccp_alpha = [0.0, 0.01, 0.02]

adaboost_clf = utils.optimize_params(estimator,
                                     X_train, y_train, cv=5,
                                     base_estimator=base_estimator,
                                     learning_rate=learning_rate,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha)

La mejor configuración de hiperparámetros seleccionada para AdaBoost es:
* `base_estimator`: 0.0
* `criterion` : entropy
* `learning_rate`: 1.0
* `base_estimator__max_depth`: 1

La tasa de acierto obtenida es de $ 0.767 \pm 0.024 $, mejora muy poco con respecto al árbol de decisión obtenido.
Obtenemos un árbol de profundidad 1, mucho sesgo pero con suficiente número de estimadores es capaz de explorar todo el espacio de búsqueda.

#### 2.2.3. Bagging

El siguiente algoritmo es *Bagging*, lo podemos encontrar en `scikit-learn` como `BaggingClassifier`.
En cuanto a los hiperparámetros algunos son similares a los utilizados en *AdaBoost*, como `base_estimator`, `n_estimators` o `random_state.

Al igual que el resto, vamos a configurar un ensemble tipo Bagging básico:

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

Partiedo del árbol de decisión obtenido vamos a optimizar el criterio de partición:

In [ ]:
estimator = bagging_model

base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]

bagging_clf = utils.optimize_params(estimator,
                                    X_train, y_train, cv=5,
                                    base_estimator=base_estimator,
                                    base_estimator__criterion=criterion)

Como podemos ver, la mejor configuración de hiperparámetros para Bagging es usar la ganancia de información (criterion="entropy") en los árboles de decisión.
La tasa de acierto obtenida es $ 0.751 \pm 0.029 $, que no mejora con respecto al anterior.

#### 2.2.4. Random Forests

Este algoritmo se trata de un meta-estimador, parecido a Bagging ya que también se busca reducir el error obtenido mediante varianza. Lo podemos encontrar en `scikit-learn` como `RandomForestClassifier`.

Al igual que el resto, vamos a configurar un ensemble tipo Random Forest básico:

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

Ahora vamos a optimizar para este algoritmo el criterio de partición (puede ser gini o entropy) y el número de características a considerar en cada nodo de los árboles de decisión (max_features).

Buscando información sobre este algoritmo, se propone para el hiperparámetro max_features usar $ \log_2{(N + 1)} $ características en cada paso de la construcción de cada árbol. En cualquier caso vamos a optar por las configuraciones que ofrece la librería por defecto: $ \sqrt{N} $,   $ \log_2{N} $ y `None`.

In [ ]:
estimator = random_forest_model

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2", None]

random_forest_clf = utils.optimize_params(estimator,
                                          X_train, y_train, cv=5,
                                          criterion=criterion,
                                          max_features=max_features)

La mejor configuración de hiperparámetros que obtenemos es para criterion gini y max_features = sqrt, teniendo una tasa de acierto de $ 0.775 \pm 0.013 $, mejora un poco con respecto a la obtenida anteriormente.

#### 2.2.5. Gradient Boosting (Gradient Tree Boosting)

Este algoritmo es una generalización de los algoritmos de Boosting con capacidad de optimizar cualquier tipo de función perdida. Lo podemos encontrar en `scikit-learn` como `GradientBoostingClassifier`.

Vamos a configurar un estimador básico utilizando los hiperparámetros por defecto:

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

Para este algoritmo vamos a optimizar el parámetro de regularización (learning_rate), el criterio de partición, la altura máxima y el parámetro de complejidad de la poda (ccp_alpha):

In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.01, 0.02]

gradient_boosting_clf = utils.optimize_params(estimator,
                                              X_train, y_train, cv=5,
                                              learning_rate=learning_rate,
                                              criterion=criterion,
                                              max_depth=max_depth,
                                              ccp_alpha=ccp_alpha)

La mejor configuración de hiperparámetros que obtenemos es para criterion friedman_mse, ccp_alpha 0.0, learning_rate 0.05 y max_depth 3, teniendo una tasa de acierto de $ 0.77 \pm 0.007 $, mejora un poco con respecto a la obtenida anteriormente.

#### 2.2.6. Histogram Gradient Boosting (Histogram-Based Gradient Boosting)

Este algoritmo es una optimización de Gradent Boosting que discretiza el conjunto de datos de entrada para reducir el número de puntos de corte a considerar en la construcción de los árboles de decisión.
El algoritmo Histogram `Gradient Boosting` se implementa en la clase `HistGradientBoostingClassifier`.

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

Para finalizar, vamos a opimizar el parámetro de regularización y el número máximo de nodos hojas de los árboles dedecisión en este algoritmo:

In [ ]:
estimator = hist_gradient_boosting_model

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [10, 20, 40, 60, 80, 100]

hist_gradient_boosting_clf = utils.optimize_params(estimator,
                                                   X_train, y_train, cv=5,
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

La mejor configuración de hiperparámetros que obtenemos es para learning_Rate 0.03 y max_leaf_nodes 10, teniendo una tasa de acierto de $ 0.771 \pm 0.03 $.

### 2.3. Selección del modelo final

Una vez obtenidos todos los modelos y sus mejores hiperparámetros para la partición de *train*, vamos a validar los resultados obtenidos y compararlos entre ellos según los resultados obtenidos por la partición de *test*.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
X = X_test
y = y_test

utils.evaluate_estimators(estimators, X, y)

Como podemos ver en los resultados obtenidos, los algoritmos no obtienen muy buena score al validar, en concreto para GradientBoosting y para RandomForests obtiene muy malos resultados.
El resto de algoritmos obtiene el mismo resultado de un 65%.

# 3. Wisconsin Breast Cancer

### 3.1. Carga de datos

Al igual que con el dataset anterior, realizamos la carga de datos.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

indexW = "id"
targetW = "diagnosis"

dataW = utils.load_data(filepath, indexW, targetW)

En este dataset el único preprocesamiento que había que realizar era eliminar la última columna, la cual no correspondía a ninguna característica ni tenía valor, por lo que procedemos a eliminarla de todo el conjunto.

In [ ]:
dataW=dataW.drop(['Unnamed: 32'], axis=1)

Comprobamos que ha sido eliminada correctamente:

In [ ]:
dataW.shape

Una vez ha sido eliminadaa, procedemos a la división del conjunto de datos:

In [ ]:
target_w = "diagnosis"

(X_w, y_w) = utils.divide_dataset(dataW, target_w)

In [ ]:
train_size = 0.7

(X_train_w, X_test_w, y_train_w, y_test_w) = train_test_split(X_w, y_w,
                                                      stratify=y_w,
                                                      train_size=train_size,
                                                      random_state=random_state)

### 3.2. Obtención de modelos

Al igual que para el anterior dataset, realizaremos los mismos pasos para obtener los mejores hiperparámetros para este problema:

#### 3.2.1. K-Nearest Neighbours (Vecinos más cercanos)

In [ ]:
n_neighbors_w = 5

k_neighbors_model_w = KNeighborsClassifier(n_neighbors_w)

In [ ]:
estimator_w = k_neighbors_model_w

weights_w = ["uniform", "distance"]
n_neighbors_w = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

k_neighbors_clf_w = utils.optimize_params(estimator_w,
                                        X_train_w, y_train_w, cv=5,
                                        weights=weights_w,
                                        n_neighbors=n_neighbors_w)

La mejor configuración de hiperparámetros que obtenemos n_neighbors 10 y weights distance, teniendo una tasa de acierto de $ 0.93 \pm 0.006 $

#### 3.2.2. Árboles de decisión

In [ ]:
decision_tree_model_w = DecisionTreeClassifier(random_state=random_state)

In [ ]:
estimator_w = clone(decision_tree_model_w)

criterion_w = ["gini", "entropy"]
max_depth_w = [1, 2, 3, 4, 5]
ccp_alpha_w = [0.0, 0.01, 0.02, 0.03, 0.04]

decision_tree_clf_w = utils.optimize_params(estimator_w,
                                          X_train_w, y_train_w, cv=5,
                                          criterion=criterion_w,
                                          max_depth=max_depth_w,
                                          ccp_alpha=ccp_alpha_w)

La mejor configuración de hiperparámetros que obtenemos es para ccp_alpha 0.01, criterio entropy y max_depth 5, teniendo una tasa de acierto de $ 0.947 \pm 0.03 $, que mejora un poco con respecto el algoritmo anterior.

#### 3.2.2. AdaBoost (Adaptative Boosting)

In [ ]:
adaboost_model_w = AdaBoostClassifier(random_state=random_state)

In [ ]:
estimator_w = adaboost_model_w

# Should not modify the base original model
base_estimator_w = clone(decision_tree_model_w)

base_estimator_w = [base_estimator_w]
learning_rate_w = [0.95, 1.0]
criterion_w = ["gini", "entropy"]
max_depth_w = [1, 2]
ccp_alpha_w = [0.0, 0.01, 0.02]

adaboost_clf_w = utils.optimize_params(estimator_w,
                                     X_train_w, y_train_w, cv=5,
                                     base_estimator=base_estimator_w,
                                     learning_rate=learning_rate_w,
                                     base_estimator__criterion=criterion_w,
                                     base_estimator__max_depth=max_depth_w,
                                     base_estimator__ccp_alpha=ccp_alpha_w)

Como podemos ver la mejor configuración de hiperparámetros que obtenemos es para:
* `base_estimator__ccp_alpha':0.0
* `base_estimator__criterion`: entropy
* `base_estimator__max_depth`: 2
* `learning_rate`: 1.0

teniendo una tasa de acierto de $ 0.98 \pm 0.017 $, ha mejorado con respecto a los dos primeros.

#### 3.2.3. Bagging

In [ ]:
bagging_model_w = BaggingClassifier(random_state=random_state)

In [ ]:
estimator_w = bagging_model_w

base_estimator_w = clone(decision_tree_model_w)

base_estimator_w = [base_estimator_w]
criterion_w = ["gini", "entropy"]

bagging_clf_w = utils.optimize_params(estimator_w,
                                    X_train_w, y_train_w, cv=5,
                                    base_estimator=base_estimator_w,
                                    base_estimator__criterion=criterion_w)

Como podemos ver, la mejor configuración de hiperparámetros para Bagging es criterion gini en los árboles de decisión.
La tasa de acierto obtenida es $ 0.952 \pm 0.026 $, que no mejora con respecto al anterior.

#### 3.2.4. Random Forests

In [ ]:
random_forest_model_w = RandomForestClassifier(random_state=random_state)

In [ ]:
estimator_w = random_forest_model_w

criterion_w = ["gini", "entropy"]
max_features_w = ["sqrt", "log2", None]

random_forest_clf_w = utils.optimize_params(estimator_w,
                                          X_train_w, y_train_w, cv=5,
                                          criterion=criterion_w,
                                          max_features=max_features_w)

La mejor configuración de hiperparámetros es criterion gini y para max_features sqrt.
La tasa de acierto obtenida es $ 0.96 \pm 0.015 $, muy similar a la obtenida con bagging.

#### 3.2.5. Gradient Boosting (Gradient Tree Boosting)

In [ ]:
gradient_boosting_model_w = GradientBoostingClassifier(random_state=random_state)

In [ ]:
estimator_w = gradient_boosting_model_w

learning_rate_w = [0.01, 0.05, 0.1]
criterion_w = ["friedman_mse", "mse"]
max_depth_w = [1, 2, 3]
ccp_alpha_w = [0.0, 0.01, 0.02]

gradient_boosting_clf_w = utils.optimize_params(estimator_w,
                                              X_train_w, y_train_w, cv=5,
                                              learning_rate=learning_rate_w,
                                              criterion=criterion_w,
                                              max_depth=max_depth_w,
                                              ccp_alpha=ccp_alpha_w)

La mejor configuración de hiperparámetros es ccp_alpha 0.0, criterion friedman_mse, learning_Rate 0.1 y max_depth 1.
La tasa de acierto obtenida es $ 0.96 \pm 0.02 $, también muy similar a las dos anteriores.

#### 3.2.6. Histogram Gradient Boosting (Histogram-Based Gradient Boosting)

In [ ]:
hist_gradient_boosting_model_w = HistGradientBoostingClassifier(random_state=random_state)

In [ ]:
estimator_w = hist_gradient_boosting_model_w

learning_rate_w = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes_w = [10, 20, 40, 60, 80, 100]

hist_gradient_boosting_clf_w = utils.optimize_params(estimator_w,
                                                   X_train_w, y_train_w, cv=5,
                                                   learning_rate=learning_rate_w,
                                                   max_leaf_nodes=max_leaf_nodes_w)

La mejor configuración de hiperparámetros es para learning_rate 0.05 y max_leaf_nodes 20.
La tasa de acierto obtenida es $ 0.965 \pm 0.025 $.

### 3.3. Selección del modelo final

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf_w,
    "Decision tree": decision_tree_clf_w,
    "AdaBoost": adaboost_clf_w,
    "Bagging": bagging_clf_w,
    "Random Forests": random_forest_clf_w,
    "Gradient Boosting": gradient_boosting_clf_w,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf_w
}

In [ ]:
X = X_test_w
y = y_test_w

utils.evaluate_estimators(estimators, X, y)

Como podemos observar en los resultados obtenidos con los diferentes algoritmos, podemos concluir que en general se obtienen buenos resultado con todos, siendo AdaBoost y Random Forests los que mejores resultados obtienen.

# 4. Análisis de un kernel de Kaggle

Navegando por los distintos conjuntos de datos de la plataforma, he seleccionado este [*kernel*](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python) para estudiarlo ya que está relacionado con lo realizado en la práctica.

Como se indica en la introducción del kernel es una básica y simple introducción a los ensembles, principalmente la variante que se conoce como Stacking, que se basa en las predicciones de otros clasificadores básicos y luego utiliza otro modelo para predecir los resultados.
El conjunto de datos empleado para esto es el del [Titanic ](https://www.kaggle.com/c/titanic).


Lo primero que realiza es importar las librerías necesarias para llevar a cabo todo el proceso.
Y una vez realizado, procede a explorar el conjunto de datos. Para ello, primero hace la carga de datos, en esste caso en la carga ya separa el conjunto de entrenamiento y de test:
* train = pd.read_csv('../input/train.csv')
* test = pd.read_csv('../input/test.csv')

Luego realiza un preprocesamiento del conjunto de datos en el que añade nuevas características y elimina otras, extrayendo la información que necesita.

El siguiente paso es visualizar el conjunto de datos en diferentes gráficas relacionando variables para extraer información.
Y en el primer gráfico podemos ver que las dos caracteríasticas que más relación tienen son family size y parch.

El siguiente paso es crear un stacking ensemble para ello se ayuda de la clase *SklearnHelper* que permite extener los métodos incorporados comunes a todos los clasficadores de Sklearn para el caso que quisieramos invocar diferentes clasificadores.

Después prepara cinco modelos de aprendizaje como clasificación de primer nivel, los utilizados son: Random Forest, Extra Trees, AdaBoost, Gradient Boosting y Support Vector Machine. Algunos han sido vistos en esta practica.
Una vez preparados, realiza el entramiento y test con los 5 clasificadores.
Ahora que se han aprendido los clasificadores de primer nivel, lo que hace es utilizar una característica de los modelos de Sklear y muestra la importancia de las diferentes características, tanto en el conjunto de entrenamiento como el de test. Esto lo hace utilizando *.featureimportances.* . Y utilizando esto crea un dataframe con los datos de importancia para luego mostrarlos gráficamente y para cada clasificador.
Esto lo utiliza para calcular la media de todas las características importantes y las almacena como una nueva columna en el dataframe y mostrarlas en un diagrama de barras.

Una vez obtenidas las predicciones del primer nivel, en el que ha construido como un nuevo conjunto de características que posteriormente se utilizarán como datos de entrenamiento para el siguiente clasificador.
Para el segundo nivel utiliza XGBoost, es una biblioteca optimizada de Gradient Boosting que ha sido diseñada para ser muy eficiente, flexible y portátil e implemente algoritmos de aprendizaje automático bajo el framework de Gradient Boosting.
Por lo que, llamad a un XGBClassifier y realiza el fit al primer nivel y a los datos del objetivo.
Finalmente, después de haber entrenado y ajustado los modelos al primer y segundo nivel, ahora ya se pueden generar las predicciones.

Como extra, añade que para mejorar se podría implementar una buena estrategia de validación cruzada en el entrenamiento de los modelos para encontrar los valores óptimos, es decir, algo similar a lo que hemos realizado en esta práctica.